In [7]:
import os 
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message) 

ChatCompletionMessage(content="In the realms of code, a pattern, fair,\nWith elegance and mystery in the air,\nThere lies a concept, both sharp and sly,\nA dance of logic, that's recursive, oh my!\n\nBehold the power, like a mirrored hall,\nA function calling itself, the mighty call,\nIt starts with a task, simple and small,\nThen branches out, expanding, standing tall.\n\nWithin the depths of this looping spell,\nIt delves deeper, like a wishing well,\nProblems divided, conquered, in stride,\nAs recursive steps gracefully collide.\n\nThrough infinite layers, it journeys on,\nLike an echo whispering through the dawn,\nA self-reflecting cycle, it abides,\nInfinite loops where the mystery hides.\n\nA hidden labyrinth, a maze of thought,\nUnwinding paths, where answers are sought,\nAs it repeats, the thread it weaves,\nUnlocking truths, as it quietly deceives.\n\nBut beware, dear programmer, tread with care,\nFor infinite recursion lurks, a snare,\nA stack overflowing, memory theft,\nLike